In [20]:
import pandas as pd
import json
import re
from lxml import html
import http.client
import threading
import urllib

In [3]:
df_french_cities = pd.read_json("french_cities.json", orient='records')

In [5]:
df_french_cities['translators_count'] = 0

In [24]:
def count_per_location(city):    
    conn = http.client.HTTPConnection("www.pagesjaunes.fr")

    headers = {
        'cache-control': "no-cache",
        'postman-token': "c7f9b543-7df3-7850-3bb2-81cfbb209da0"
        }

    conn.request("GET", "/annuaire/chercherlespros?quoiqui=Traduction&ou={ou}&proximite=0".format(ou=urllib.parse.quote(city)), headers=headers)
    
    try:
        res = conn.getresponse()
        data = res.read()
        body = data.decode("latin-1")

        tree = html.fromstring(body)
        elem = tree.xpath('//strong[@id="SEL-nbresultat"]/text()')
        count = int(elem[0].replace(' ', ''))
    except Exception as e:
        print(e)
        print(body)
        if 'elem' in locals():
            print(elem)
        count = 0
    
    return count

In [25]:
for idx, elm in df_french_cities.iterrows():
    df_french_cities.loc[idx, 'translators_count'] = count_per_location(elm['city'])
    completion = 100 * (idx+1)/df_french_cities.shape[0]
    print("\r{0:.2f}% ... {city} ... {count} Translators.".format(
            completion,
            city=elm['city'],
            count=df_french_cities['translators_count'].sum()
        ), end="")

100.00% ... Thionville ... 4057 Translators.

In [27]:
df_french_cities.to_json("french_cities_translators_count.json", orient="records")